In [363]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from joblib import Parallel, delayed
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [364]:
#read the dataset and display the first few rows
df = pd.read_csv('sales.csv')
df.head()

,d,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,sales
0,425390,366,4,2013-04-18,517,1,0,0,0,4422
1,291687,394,6,2015-04-11,694,1,0,0,0,8297
2,411278,807,4,2013-08-29,970,1,1,0,0,9729
3,664714,802,2,2013-05-28,473,1,1,0,0,6513
4,540835,726,4,2013-10-10,1068,1,1,0,0,10882


In [365]:
#missing values
print(df.isnull().sum())

d                      0
store_ID               0
day_of_week            0
date                   0
nb_customers_on_day    0
open                   0
promotion              0
state_holiday          0
school_holiday         0
sales                  0
dtype: int64


In [366]:
#printing data types
print(df.dtypes)

d                       int64
store_ID                int64
day_of_week             int64
date                   object
nb_customers_on_day     int64
open                    int64
promotion               int64
state_holiday          object
school_holiday          int64
sales                   int64
dtype: object


In [367]:
#dropping object type columns
df = df.drop(columns=["d", "date"])

In [368]:
# Transforming categorical variable into numerical
df['state_holiday'] = LabelEncoder().fit_transform(df['state_holiday'])
print(df.dtypes)

store_ID               int64
day_of_week            int64
nb_customers_on_day    int64
open                   int64
promotion              int64
state_holiday          int64
school_holiday         int64
sales                  int64
dtype: object


In [369]:
#Train-Test-Split initialization
X = df.drop(columns="sales") # sending all columns except sales to X
y = df['sales'] # target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(512672, 7) (128168, 7) (512672,) (128168,)


In [370]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=30, max_depth=4, n_jobs=-1),
    # "KNN Regressor": KNeighborsRegressor(n_neighbors=5),
    # "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Step 2: Function to train and evaluate a model
def train_and_evaluate(name, model, X_train, y_train, X_test, y_test):
    X_train = df.drop(columns="sales") # sending all columns except sales to X
    y_train = df['sales'] # target variable
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score = r2_score(y_test, predictions)
    return name, score, predictions

# Step 3: Run all models in parallel
def run_models_parallel(X_train, y_train, X_test, y_test):
    results = Parallel(n_jobs=3)(
        delayed(train_and_evaluate)(name, model, X_train, y_train, X_test, y_test)
        for name, model in models.items()
    )
    return results

In [371]:
#Bagging Technique
# Run the models and get results
results = run_models_parallel(X_train, y_train, X_test, y_test)

# Step 5: Print R2 scores
for name, score, _ in results:
    print(f"{name} R2 Score: {score:.4f}")


Linear Regression R2 Score: 0.8507
Random Forest R2 Score: 0.8426
Gradient Boosting R2 Score: 0.8868


In [372]:
# Step 6: Create bagged predictions (average of predictions)
all_preds = np.array([preds for _, _, preds in results])
bagged_preds = np.mean(all_preds, axis=0)

In [373]:
# Step 7: Add bagged predictions as a new column in test data
X_test_with_preds = X_test.copy()
X_test_with_preds['Predicted_Sales_Bagged'] = bagged_preds
bagged_r2 = r2_score(y_test, bagged_preds)

# Optional: Show some results
print(X_test_with_preds[['Predicted_Sales_Bagged']].head())
print('R2 Score for Bagged:', bagged_r2)

        Predicted_Sales_Bagged
360263             9638.923090
226238              -44.728937
94660              4697.839793
322601             4911.901926
450235             5775.627905
R2 Score for Bagged: 0.8694452676397693


In [374]:
#identify categorical columns (if any remain after drop)
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
#fill missing numerical values with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in num_cols:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_32238/4003619317.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)


In [ ]:


# Load real-life data
df_real = pd.read_csv("REAL_DATA.csv")


def preprocess_data(df, drop_cols, num_cols, cat_cols, reference_cols=None):
    # Drop columns
    df = df.drop(drop_cols, axis=1, errors='ignore')
    
    # Fill missing numeric values with median
    for col in num_cols:
        if col in df.columns:
            df[col].fillna(df[col].median(), inplace=True)
            
    # Fill missing categorical values with mode
    for col in cat_cols:
        if col in df.columns:
            df[col].fillna(df[col].mode()[0], inplace=True)
    
    # Encode categorical columns
    for col in cat_cols:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix=col)
            df = pd.concat([df, dummies], axis=1)
            df.drop(col, axis=1, inplace=True)
    
    # Align columns with reference dataframe (e.g. X_train)
    if reference_cols is not None:
        missing_cols = set(reference_cols) - set(df.columns)
        for col in missing_cols:
            df[col] = 0
        df = df[reference_cols]  # reorder columns
    return df

#define columns info used in training
drop_cols = ['state_holiday', 'date']  # make sure column names match

#preprocess real data to match training data format
df_real_processed = preprocess_data(df_real, drop_cols, num_cols, cat_cols, reference_cols=X_train.columns)

#predict sales using the trained model
sales_pred = ?????.predict(df_real_processed)

#new dataframe with real data and predictions
df_real_processed['sales'] = sales_pred


/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_32238/2404762624.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


AttributeError: 'list' object has no attribute 'predict'

Observations:

